In [1]:
def president_home_state(president,state_options):
    
    print("Hello! Let's test your knowledge of the home states of US Presidents!")
    print(f"In what state was this president born: {president}\n")
    
    for num,option in enumerate(state_options):
        print('\n')
        print(f"Definition #{num} is: {option}")
    
    print('\n')
    num_choice = input("What is your choice? (Return the single number option)")
    
    return state_options[int(num_choice)]
    

In [ ]:
# president_home_state("George Washington", ["Virginia", "New York", "Massachusetts"])

In [2]:
function_json = {'type':'function',
            'function':{
                'name': 'president_home_state',
                'parameters':{
                    "type":"object",
                      "properties":{
                          "president": {'type':'string','description':"The name of a random US President"},
                          "state_options":{'type':'array',
                                        'items':{'type':'string'},
                                        'description':"A Python list of strings of US States, where one of the states is the birthplace state of the US President."}
                      },
                    'required' : ["president","state_options"]
                }

    }
} 

In [3]:
from openai import OpenAI


In [4]:
client = OpenAI()

In [5]:
assistant = client.beta.assistants.create(name='US Presidents Quiz Bot',
  instructions="You help create a quiz where you give a US President and a list of birthplace states, where only one is the correct birthplace state of the president. Later you check if answers returned are correct.",
  model="gpt-3.5-turbo",
  tools=[function_json]
)

In [6]:
thread = client.beta.threads.create()

In [7]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create a new quiz question with a US President and a list of options for the home state of birth. Then I will reply with a single state and let me know if I got it right.",
)

In [8]:
run = client.beta.threads.runs.create(thread_id=thread.id,assistant_id=assistant.id)

In [9]:
run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)

In [10]:
run.status

'requires_action'

In [11]:
import json

In [12]:
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

In [13]:
arguments

{'president': 'Abraham Lincoln',
 'state_options': ['Kentucky', 'Illinois', 'Ohio', 'Virginia']}

In [14]:
name

'president_home_state'

In [15]:
response = president_home_state(arguments['president'],arguments['state_options'])

Hello! Let's test your knowledge of the home states of US Presidents!
In what state was this president born: Abraham Lincoln



Definition #0 is: Kentucky


Definition #1 is: Illinois


Definition #2 is: Ohio


Definition #3 is: Virginia




What is your choice? (Return the single number option) 1


In [16]:
response

'Illinois'

In [17]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {"tool_call_id": tool_call.id,
            "output": json.dumps(response),
        }
    ],
)

In [18]:
# run

In [19]:
messages = client.beta.threads.messages.list(thread.id)

In [20]:
for thread_message in messages:
    print(thread_message.content[0].text.value)
    print('\n')

Was Abraham Lincoln born in Illinois?
- Yes, that's correct! Abraham Lincoln was born in Illinois.


Create a new quiz question with a US President and a list of options for the home state of birth. Then I will reply with a single state and let me know if I got it right.




In [21]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
response = client.beta.assistants.delete(my_assistants.data[0].id)